<a href="https://colab.research.google.com/github/L12161/UWMMSE-Power-alloc/blob/main/in_tf2_no_py_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA GENERATION SECTION

In [ ]:
# !pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
# import os
# import sys
# import pdb      # debugg korte help korbe ; aro jante:  https://realpython.com/python-debugging-pdb/
# import pickle   # python object serializer. Eta diee ami kono obj, list, table ke file hishebe save kore felte parbo
# # aro jante : https://docs.python.org/3/library/pickle.html 
# import numpy as np


In [ ]:
import shutil
SOURCE_PATH = '/content/SISO_DATASET'
DESTINATION_PATH = '/content/gdrive/My Drive/THESIS DATASETS'
shutil.move(SOURCE_PATH, DESTINATION_PATH)

'/content/gdrive/My Drive/THESIS DATASETS/SISO_DATASET'

In [ ]:
transmitters_fix = transmitters
receivers_fix = receivers

In [ ]:
import numpy as np

nNodes = 20 
pl = 2.2
alpha = 1
r=1 

# transmitters = np.random.uniform(low=-nNodes/r, high=nNodes/r, size=(nNodes,2))
# receivers = transmitters + np.random.uniform(low=-nNodes/4,high=nNodes/4, size=(nNodes,2))
transmitters = transmitters_fix
receivers = receivers_fix
L = np.zeros((nNodes,nNodes))

for i in np.arange(nNodes):
  for j in np.arange(nNodes):
    d = np.linalg.norm(transmitters[i,:]-receivers[j,:])
    L[i,j] = np.power(d,-pl)


In [ ]:
# Datagen.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import pdb
import pickle
import numpy as np

# Eperiment
dataID = 'nNode_25_pl_2p2'

# Number of nodes
nNodes = 25

# Path gain exponent
pl = 2.2

# Rayleigh distribution scale
alpha = 1 

# Batch size
batch_size = 64

# Training iterations
tr_iter = 10000

# Testing iterations
te_iter = 100


# Build random geometric graph
def build_adhoc_network( nNodes, r=1, pl=2.2 ):
    transmitters = np.random.uniform(low=-nNodes/r, high=nNodes/r, size=(nNodes,2))
    receivers = transmitters + np.random.uniform(low=-nNodes/4,high=nNodes/4, size=(nNodes,2))
    # transmitters = transmitters_fix
    # receivers = receivers_fix

    L = np.zeros((nNodes,nNodes))

    for i in np.arange(nNodes):
        for j in np.arange(nNodes):
            d = np.linalg.norm(transmitters[i,:]-receivers[j,:])
            L[i,j] = np.power(d,-pl)

    return( dict(zip(['tx', 'rx'],[transmitters, receivers] )), L )

# Simulate Fading
def sample_graph(batch_size, A, nNodes, alpha=1.):
    samples = np.random.rayleigh(alpha, (batch_size, nNodes, nNodes))
    #samples = (samples + np.transpose(samples,(0,2,1)))/2
    PP = samples[None,:,:] * A
    return PP[0]

# Training Data
def generate_data(batch_size, alpha, A, nNodes):
    tr_H = []
    te_H = []
    
    for indx in range(tr_iter):
        # sample training data 
        H = sample_graph(batch_size, A, nNodes, alpha )
        tr_H.append( H )

    for indx in range(te_iter):
        # sample test data 
        H = sample_graph(batch_size, A, nNodes, alpha )
        te_H.append( H )

    return( dict(zip(['train_H', 'test_H'],[tr_H, te_H] ) ) )
        
def main():
    coord, A = build_adhoc_network( nNodes )
    
    # Create data path
    if not os.path.exists('data/'+dataID):
        os.makedirs('data/'+dataID)

    # Coordinates of nodes
    f = open('data/'+dataID+'/coordinates.pkl', 'wb')  
    pickle.dump(coord, f)         
    f.close()

    # Geometric graph
    f = open('data/'+dataID+'/A.pkl', 'wb')  
    pickle.dump(A, f)          
    f.close()
    
    # Training data
    data = generate_data(batch_size, alpha, A, nNodes)
    f = open('data/'+dataID+'/H.pkl', 'wb')  
    pickle.dump(data, f)         
    f.close()

    

if __name__ == '__main__':
    rn = np.random.randint(2**20)
    rn1 = np.random.randint(2**20)
    np.random.seed(rn1)

    main()

In [ ]:
# coord, A_dummy = build_adhoc_network( nNodes )
# H_hopefullyCH = sample_graph(64, A_dummy, nNodes, 1 )
# len(H_hopefullyCH)

# Model gen

In [ ]:
import pdb
import tensorflow as tf

# UWMMSE
class UWMMSE(object):
        # Initialize
        def __init__( self, Pmax=1., var=7e-10, feature_dim=3, batch_size=64, layers=4, learning_rate=1e-3, max_gradient_norm=5.0, exp='uwmmse' ):
            self.Pmax              = tf.cast( Pmax, tf.float64 )
            self.var               = var
            self.feature_dim       = feature_dim
            self.batch_size        = batch_size
            self.layers            = layers
            self.learning_rate     = learning_rate
            self.max_gradient_norm = max_gradient_norm
            self.exp               = exp
            self.global_step = tf.Variable(0, trainable=False, name='global_step')
            self.build_model()

        # Build Model
        def build_model(self):
            self.init_placeholders()
            self.build_network()
            self.build_objective()
            
        def init_placeholders(self):
            # CSI [Batch_size X Nodes X Nodes]
            self.H = tf.compat.v1.placeholder(tf.float64, shape=[None, None, None], name="H")
            
            # NSI [Batch_size X Nodes X Features]
            #self.x = tf.compat.v1.placeholder(tf.float64, shape=[None, None, self.feature_dim], name="x")
            
            # Node Weights [Batch_size X Nodes]
            #self.alpha = tf.compat.v1.placeholder(tf.float64, shape=[None, None], name="alpha")
            
            # Boolean for Training/Inference 
            #self.phase = tf.compat.v1.placeholder_with_default(False, shape=(), name='phase')
        
        
        # Building network
        def build_network(self):
            # Squared H 
            self.Hsq = tf.math.square(self.H)
            
            # Diag H
            dH =  tf.linalg.diag_part( self.H ) 
            self.dH = tf.linalg.diag( dH )
            
            # Retrieve number of nodes for initializing V
            self.nNodes = tf.shape( self.H )[-1]

            # Maximum V = sqrt(Pmax)
            Vmax = tf.math.sqrt(self.Pmax)

            # Initial V
            V = Vmax * tf.ones([self.batch_size, self.nNodes], dtype=tf.float64)
            
            self.pow_alloc = []
            
            # Iterate over layers l
            for l in range(self.layers):
                with tf.compat.v1.variable_scope('Layer{}'.format(l+1)):
                    # Compute U^l
                    U = self.U_block( V )
                    
                    # Compute W^l
                    W_wmmse = self.W_block( U, V )
                    
                    # Learn a^l
                    #print("a = ")
                    a = self.gcn('a')
                    #print(tf.shape(a))
                    #a_value = a.eval(session=tf.compat.v1.Session())
                    #print(a_value)

                    # Learn b^l
                    #print("b = ")
                    b = self.gcn('b')
                    #print(tf.shape(b))
                    #b_value = b.eval(session=tf.compat.v1.Session())
                    #print(b_value)

                    # Compute Wcap^l = a^l * W^l + b^l
                    W = tf.math.add( tf.math.multiply( a, W_wmmse ), b )
                    
                    # Learn mu^l
                    mu = tf.compat.v1.get_variable( name='mu', initializer=tf.constant(0., shape=(), dtype=tf.float64))

                    # Compute V^l
                    if self.exp == 'wmmse':
                        V = self.V_block( U, W_wmmse, 0. )
                    else:
                        V = self.V_block( U, W, mu )
                    
                    # Saturation non-linearity  ->  0 <= V <= Vmax
                    V = tf.math.minimum(V, Vmax) + tf.math.maximum(V, 0) - V

            # Final V
            self.pow_alloc = V
        
        def U_block(self, V):
            # H_ii * v_i
            num = tf.math.multiply( tf.linalg.diag_part(self.H), V )
            
            # sigma^2 + sum_j( (H_ji)^2 * (v_j)^2 )
            den = tf.reshape( tf.matmul( tf.transpose( self.Hsq, perm=[0,2,1] ), tf.reshape( tf.math.square( V ), [-1, self.nNodes, 1] ) ), [-1, self.nNodes] ) + self.var 
            
            # U = num/den
            return( tf.math.divide( num, den ) )

        # Sum-rate = z
        def W_block(self, U, V):
            # 1 - u_i * H_ii * v_i
            den = 1. - tf.math.multiply( tf.linalg.diag_part(self.H), tf.math.multiply( U, V ) )
            
            # W = 1/den
            return( tf.math.reciprocal( den ) )

        # Weighted Sum-rate = a * z
        def W_block1(self, U, V):
            # 1 - u_i * H_ii * v_i
            den = 1. - tf.math.multiply( tf.linalg.diag_part(self.H), tf.math.multiply( U, V ) )
            
            # W = alpha/den
            return( tf.math.divide( self.alpha, den ) )        


        def gcn(self, name):
            # 2 Layers
            L = 2
            
            # Hidden dim = 5
            input_dim = [self.feature_dim,5]
            output_dim = [5,1]
            
            ## NSI [Batch_size X Nodes X Features]
            x = tf.ones([self.batch_size, self.nNodes, 1], dtype=tf.float64)
            #x = self.x
                        
            with tf.compat.v1.variable_scope('gcn_'+name):
                for l in range(L):
                    with tf.compat.v1.variable_scope('gc_l{}'.format(l+1)):
                        # Weights
                        w1 = tf.compat.v1.get_variable( name='w1', shape=(input_dim[l], output_dim[l]), initializer=tf.initializers.glorot_uniform(), dtype=tf.float64)
                        w0 = tf.compat.v1.get_variable( name='w0', shape=(input_dim[l], output_dim[l]), initializer=tf.initializers.glorot_uniform(), dtype=tf.float64)
                        
                        ## Biases
                        b1 = tf.compat.v1.get_variable( name='b1', initializer=tf.constant(0.1, shape=(output_dim[l],), dtype=tf.float64) )
                        b0 = tf.compat.v1.get_variable( name='b0', initializer=tf.constant(0.1, shape=(output_dim[l],), dtype=tf.float64) )
                        
                        # XW
                        x1 = tf.matmul(x, w1)
                        x0 = tf.matmul(x, w0)
                        
                        # diag(A)XW0 + AXW1
                        x1 = tf.matmul(self.H, x1)  
                        x0 = tf.matmul(self.dH, x0)
                        
                        ## AXW + B
                        x1 = tf.add(x1, b1)
                        x0 = tf.add(x0, b0)
                        
                        # Combine
                        x = x1 + x0
                        
                        # activation(AXW + B)
                        if l == 0:
                            x = tf.nn.relu(x)  
                        else:
                            x = tf.nn.sigmoid(x)

                # Coefficients (a / b) [Batch_size X Nodes]
                output = tf.squeeze(x)
                #print(name)
                #output
            
            return output
        
        def V_block(self, U, W, mu):
            # H_ii * u_i * w_i
            num = tf.math.multiply( tf.linalg.diag_part(self.H), tf.math.multiply( U, W ) )
            
            # mu + sum_j( (H_ij)^2 * (u_j)^2 *w_j )
            den = tf.math.add( tf.reshape( tf.matmul( self.Hsq, tf.reshape( tf.math.multiply( tf.math.square( U ), W ), [-1, self.nNodes, 1] ) ), [-1, self.nNodes] ), mu)
            
            # V = num/den
            return( tf.math.divide( num, den ) )        
                                                                                
        def build_objective(self):
            # (H_ii)^2 * (v_i)^2
            num = tf.math.multiply( tf.linalg.diag_part(self.Hsq), tf.math.square( self.pow_alloc ) )
            
            # sigma^2 + sum_j j ~= i ( (H_ji)^2 * (v_j)^2 ) 
            den = tf.reshape( tf.matmul( tf.transpose( self.Hsq, perm=[0,2,1] ), tf.reshape( tf.math.square( self.pow_alloc ), [-1, self.nNodes, 1] ) ), [-1, self.nNodes] ) + self.var - num 
            
            # rate
            rate = tf.math.log( 1. + tf.math.divide( num, den ) ) / tf.cast( tf.math.log( 2.0 ), tf.float64 )
            
            # Sum Rate = sum_i ( log(1 + SINR) )
            self.utility = tf.reduce_sum( rate, axis=1 )
            
            # Weighted Sum Rate
            #rate = tf.math.multiply( self.alpha, rate )
            #self.utility = tf.reduce_sum( rate, axis=1 )
            
            # Minimization objective
            self.obj = -tf.reduce_mean( self.utility )
            
            if self.exp == 'uwmmse':
                self.init_optimizer()

        def init_optimizer(self):
            # Gradients and SGD update operation for training the model
            self.trainable_params = tf.compat.v1.trainable_variables()
            
            #Learning Rate Decay
            #starter_learning_rate = self.learning_rate
            #self.learning_rate_decayed = tf.train.exponential_decay(starter_learning_rate, global_step=self.global_step, decay_steps=5000, decay_rate=0.99, staircase=True)
            
            # SGD with Momentum
            #self.opt = tf.train.GradientDescentOptimizer( learning_rate=learning_rate )
            #self.opt = tf.train.MomentumOptimizer(learning_rate=self.learning_rate_decayed, momentum=0.9, use_nesterov=True )

            # Adam Optimizer
            self.opt = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)

            # Compute gradients of loss w.r.t. all trainable variables
            gradients = tf.gradients(self.obj, self.trainable_params)

            # Clip gradients by a given maximum_gradient_norm
            clip_gradients, _ = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
            
            # Update the model
            self.updates = self.opt.apply_gradients(
                zip(clip_gradients, self.trainable_params), global_step=self.global_step)
                
        def save(self, sess, path, var_list=None, global_step=None):
            saver = tf.compat.v1.train.Saver(var_list)
            save_path = saver.save(sess, save_path=path, global_step=global_step)

        def restore(self, sess, path, var_list=None):
            saver = tf.compat.v1.train.Saver(var_list)
            saver.restore(sess, save_path=tf.train.latest_checkpoint(path))

        def train(self, sess, inputs ):
            input_feed = dict()
            input_feed[self.H.name] = inputs
            #input_feed[self.x.name] = features
            #input_feed[self.alpha.name] = alpha
            
            # Training Phase
            #input_feed[self.phase.name] = True
 
            output_feed = [self.obj, self.utility, self.pow_alloc, self.updates]
                            
            outputs = sess.run(output_feed, input_feed)
            
            return outputs[0], outputs[1], outputs[2]


        def eval(self, sess, inputs ):
            input_feed = dict()
            input_feed[self.H.name] = inputs
            #input_feed[self.x.name] = features
            #input_feed[self.alpha.name] = alpha

            # Training Phase
            #input_feed[self.phase.name] = False

            output_feed = [self.obj,self.utility, self.pow_alloc] 
                           
            outputs = sess.run(output_feed, input_feed)
            
            return outputs[0], outputs[1], outputs[2]


# MAIN 

In [ ]:
argv_3 = 'train'
argv_2 = 'uwmmse'
argv_1 = 'nNode_25_pl_2p2'

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import sys
import pdb
import time
import math
import pickle
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)


# Experiment 
dataID = argv_1
exp = argv_2
mode = argv_3
if len( sys.argv ) > 3:
    #mode = sys.argv[3]
    mode = argv_3

# Maximum available power at each node
Pmax = 1.0

# Noise power
var_db = -91
var = 10**(var_db/10)

# Features
feature_dim = 1

# Batch size
batch_size = 64

# Layers UWMMSE = 4 (default)  WMMSE = 100 (default)
layers = 4 if exp == 'uwmmse' else 100

# Learning rate
learning_rate=1e-3

# Number of epochs
nEpoch = 20

    
# Create Model Instance
def create_model( session, exp='uwmmse' ):
    # Create
    model = UWMMSE( Pmax=Pmax, var=var, feature_dim=feature_dim, batch_size=batch_size, layers=layers, learning_rate=learning_rate, exp=exp )
    
    # Initialize variables ( To train from scratch )
    session.run(tf.compat.v1.global_variables_initializer())
    
    return model

# Train
def mainTrain():        
    # Data
    # uncomment this on your first runs. This line loads the dataset. If your dataset loads multiple times, the ram will overflow
    H = pickle.load( open( '/content/gdrive/My Drive/THESIS DATASETS/SISO_DATASET/'+dataID+'/H.pkl', 'rb' ) )
    
    #Training data
    train_H = H['train_H']
    
    #Test data
    test_H = H['test_H']
    
    # Initiate TF session
    with tf.compat.v1.Session(config=config) as sess:
    
        # WMMSE experiment
        if exp == 'wmmse':
        
            # Create model 
            model = create_model( sess, exp )
            
            # Test
            test_iter = len(test_H)
                    
            print( '\nWMMSE Started\n' )

            t = 0.
            test_rate = 0.0
            sum_rate = []
            
            for batch in range(test_iter):
                #print(batch)
                batch_test_inputs = test_H[batch]                
                start = time.time()
                avg_rate, batch_rate, batch_power = model.eval( sess, inputs=batch_test_inputs )
                t += (time.time() - start)
                test_rate += -avg_rate
                sum_rate.append( batch_rate )
                
            test_rate /= test_iter

            # Average per-iteration test time
            t = t / test_iter
            
            log = "Test_rate = {:.3f}, Time = {:.3f} sec\n"
            print(log.format( test_rate, t))
            
        # Unrolled WMMSE experiment
        else:
            
            # Create model 
            model = create_model( sess )

            if mode == 'train':
                # Create model path
                if not os.path.exists('models/'+dataID):
                    os.makedirs('models/'+dataID)
                    
                #Training loop
                print( '\nUWMMSE Training Started\n' )
                max_rate = 0.
                train_iter = len(train_H)
                
                #nEpoch = 1
                for epoch in range(nEpoch):
                    start = time.time()
                    train_rate = 0.0
                
                    for it in range(train_iter):
                        batch_train_inputs = train_H[it]
                        step_rate, batch_rate, power  = model.train( sess, inputs=batch_train_inputs ) 
                        if np.isnan(step_rate) or np.isinf(step_rate) :
                            pdb.set_trace()
                        train_rate += -step_rate
                    train_rate /= train_iter
                    
                    log = "Epoch {}/{}, Average Sum_rate = {:.3f}, Time = {:.3f} sec\n"
                    print(log.format( epoch+1, nEpoch, train_rate, time.time() - start) )
                    
                    # Save model with best average sum-rate
                    if train_rate > max_rate:
                        max_rate = train_rate
                        model.save(sess, path='models/'+dataID+'/uwmmse-model', global_step=(epoch+1))
                    
                    # Shuffle
                    shuffled_indices = np.random.permutation(train_iter)
                    train_H = [train_H[indx] for indx in shuffled_indices]

                    
                print( 'Training Complete' )

            # Test
            test_iter = len(test_H)       

            # Restore best saved model
            model.restore(sess, path='models/'+dataID+'/')

            print( '\nUWMMSE Testing Started\n' )

            t = 0.
            test_rate = 0.0
            sum_rate = []

            for batch in range(test_iter):
                batch_test_inputs = test_H[batch]
                start = time.time()
                avg_rate, batch_rate, batch_power = model.eval( sess, inputs=batch_test_inputs )
                if np.isnan(avg_rate) or np.isinf(avg_rate):
                    pdb.set_trace()
                t += (time.time() - start)
                sum_rate.append( batch_rate )
                test_rate += -avg_rate
            
            
            test_rate /= test_iter
            
            ## Average per-iteration test time   
            t = t / test_iter

            log = "Test_rate = {:.3f}, Time = {:.3f} sec\n"
            print(log.format( test_rate, t))
            #print(sum_rate)

            
if __name__ == "__main__":        
    import sys

    rn = np.random.randint(2**20)
    rn1 = np.random.randint(2**20)
    tf.random.set_seed(rn)
    np.random.seed(rn1)

    mainTrain()



UWMMSE Training Started

Epoch 1/20, Average Sum_rate = 106.110, Time = 112.621 sec

Epoch 2/20, Average Sum_rate = 108.026, Time = 101.393 sec

Epoch 3/20, Average Sum_rate = 108.185, Time = 101.225 sec

Epoch 4/20, Average Sum_rate = 108.234, Time = 101.240 sec

Epoch 5/20, Average Sum_rate = 108.268, Time = 99.581 sec

Epoch 6/20, Average Sum_rate = 108.304, Time = 100.667 sec

Epoch 7/20, Average Sum_rate = 108.333, Time = 105.050 sec

Epoch 8/20, Average Sum_rate = 108.353, Time = 100.902 sec

Epoch 9/20, Average Sum_rate = 108.369, Time = 100.143 sec

Epoch 10/20, Average Sum_rate = 108.386, Time = 99.502 sec

Epoch 11/20, Average Sum_rate = 108.406, Time = 102.253 sec

Epoch 12/20, Average Sum_rate = 108.428, Time = 102.061 sec

Epoch 13/20, Average Sum_rate = 108.452, Time = 102.242 sec

Epoch 14/20, Average Sum_rate = 108.477, Time = 100.848 sec

Epoch 15/20, Average Sum_rate = 108.503, Time = 100.537 sec

Epoch 16/20, Average Sum_rate = 108.527, Time = 99.974 sec

Epoch 17/2

# just uwmmse test

# When you need to clear it all and make another run

In [ ]:
sys.modules[__name__].__dict__.clear()